This notebook is used to show the approach to the data deduplication using LSH (Locality Sensitive Hashing).

Installing dependencies

In [1]:
! pip install -r requirements.txt

In [3]:
from datasketch import MinHash, MinHashLSH
import os
import itertools
import re
from tqdm import tqdm
import subprocess

Load a sample of the data

In [4]:
folder_path = './start_data'

result = subprocess.run(["find", folder_path, "-maxdepth", "1", "-type", "f"], capture_output=True, text=True)
files = result.stdout.split('\n')

python(46375) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [9]:
def normalize_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def tokenize_text(text):
    return re.findall(r'\b\w+\b', text.lower())


def deduplicate_text_files_lsh(folder_path, threshold=0.5, num_perm=128, limit=None):
    duplicates = []
    unique_files = []

    original_to_duplicates = {}
    with os.scandir(folder_path) as entries:
        lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
        processed_count = 0
        unique_files_count = 0
        for file_path in entries:
            try:
                # file_path = os.path.join(folder_path, file)
                file_content = open(file_path, 'r').read()
                minhash = MinHash(num_perm=num_perm)
                for word in tokenize_text(normalize_text(file_content)):
                    minhash.update(word.encode('utf-8'))
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
                continue
            
            result = lsh.query(minhash)
            if result:
                canonical = result[0]
                original_to_duplicates.setdefault(canonical, []).append(file_path)
                duplicates.append(file_path)
            else:
                lsh.insert(file_path, minhash)
                unique_files.append(file_path)
                unique_files_count += 1
            processed_count += 1
            if processed_count % 1000 == 0:
                print(f"Processed {processed_count} files")

    return duplicates, unique_files, original_to_duplicates

# thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
thresholds = [0.8]

for threshold in thresholds:
    dupicates, unique_files, original_to_duplicates = deduplicate_text_files_lsh(folder_path, threshold=threshold)
    print(f'Threshold: {threshold}')
    print(len(dupicates)/len(unique_files+dupicates) * 100)

4857200


In [21]:
dupicates, unique_files, original_to_duplicates = deduplicate_text_files_lsh(folder_path, threshold=0.8)
print(original_to_duplicates)
print(len(dupicates), len(unique_files))


Processing 48572 files


100%|██████████| 48572/48572 [04:46<00:00, 169.80it/s]

{'116464767.txt': ['116464773.txt', '116464770.txt', '116464758.txt', '116464749.txt', '116464774.txt', '116464762.txt', '116464776.txt', '116449005.txt', '116449010.txt', '116449004.txt', '116464753.txt', '116464747.txt', '116464751.txt', '116464745.txt', '116464779.txt', '116449007.txt', '116464744.txt', '116464754.txt', '116464769.txt', '116464757.txt', '116449000.txt'], '116453691.txt': ['116453685.txt', '116453684.txt', '116453690.txt', '116453686.txt', '116453692.txt', '116489148.txt', '116453687.txt', '116453697.txt', '116427229.txt', '116427230.txt', '116427231.txt', '116427233.txt', '116427232.txt', '116489151.txt', '116453689.txt', '116453688.txt'], '116456716.txt': ['116456702.txt', '116456680.txt'], '116451051.txt': ['116485566.txt', '116450206.txt', '116450947.txt', '116450942.txt', '116451123.txt', '116451056.txt', '116450941.txt', '116451121.txt', '116451192.txt', '116451193.txt', '116450934.txt', '116451209.txt', '116451195.txt', '116450079.txt', '116450938.txt', '11645

In [23]:
! cat start_data/116464767.txt





Провадження № 2/712/967/24
Справа № 712/13322/23

У Х В А Л А 

про відкриття провадження у справі


 
23 січня 2024 року Суддя Соснівського районного суду м. Черкаси Пироженко С.А., розглянувши позовну заяву Комунального підприємства теплових мереж «Черкаситеплокомуненерго» Черкаської міської ради (адреса: м. Черкаси, вул. О. Дашковича 62) до ОСОБА_1 , ОСОБА_2 (адреса: АДРЕСА_1 ) про стягнення боргу за спожиту теплову енергію,

В С Т А Н О В И В:

Позивач звернувся з позовом до відповідачів про стягнення боргу за спожиту теплову енергію.
Позовна заява подана з додержанням вимог ст.ст. 175-177 ЦПК України.
З огляду на предмет позову дана справа підпадає під ознаки малозначної справи та не віднесена до категорії справ, які підлягають розгляду лише за правилами загального позовного провадження. Обставини справи, які згідно частини 3 статті 274 ЦПК України мають значення для вирішення питання про можливість розгляду справи в порядку спрощеного провадження, також свідчать про наявність

In [24]:
! cat start_data/116464762.txt





Провадження № 2/712/973/24
Справа № 712/13386/23

У Х В А Л А 

про відкриття провадження у справі


 
23 січня 2024 року Суддя Соснівського районного суду м. Черкаси Пироженко С.А., розглянувши позовну заяву Комунального підприємства теплових мереж «Черкаситеплокомуненерго» Черкаської міської ради (адреса: м. Черкаси, вул. О. Дашковича 62) до ОСОБА_1 (адреса: АДРЕСА_1 ) про стягнення боргу за спожиту теплову енергію,

В С Т А Н О В И В:

Позивач звернувся з позовом до відповідача про стягнення боргу за спожиту теплову енергію.
Позовна заява подана з додержанням вимог ст.ст. 175-177 ЦПК України.
З огляду на предмет позову дана справа підпадає під ознаки малозначної справи та не віднесена до категорії справ, які підлягають розгляду лише за правилами загального позовного провадження. Обставини справи, які згідно частини 3 статті 274 ЦПК України мають значення для вирішення питання про можливість розгляду справи в порядку спрощеного провадження, також свідчать про наявність підстав дл